In [1]:
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.enums import Resampling
from rasterio.transform import from_origin, Affine
from rasterio.mask import mask
import geopandas as gpd
from scipy.interpolate import griddata
from osgeo import gdal, gdalconst
import xdem
import matplotlib.pyplot as plt

### 1. Loading and inspecting raster data with rasterio

To explore how raster data is stored in python when it is loaded with rasterio by answering the following questions:

1) What is stored in dem_src, what is stored in dem2020?
2) Explore the metadata. What are the dimensions, coordinate reference system, and resolution of this dataset?
3) Can you also derive the no-data value, and the elevation range of the DEM?
4) Plot the 2020 DEM (define some reasonable minimum and maximum values). What do you notice about the axes? Where in space is this data?
5) Plot the data again, passing plt.imshow() the correct extent.

In [2]:
#load with rasterio
with rasterio.open('./data/DEMs/Silvretta-2020-2m.tif') as dem_src:
    dem2020 = dem_src.read(1)

In [8]:
# What datatypes are dem_src and dem2020, and what does dem2020 contain?
type(...)

ellipsis

In [9]:
# Explore metadata
# What is the file resolution, the image dimensions, the spatial resolution, and the no-data value?
dem_src.meta
 

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.4028234663852886e+38,
 'width': 4623,
 'height': 3312,
 'count': 1,
 'crs': CRS.from_epsg(2056),
 'transform': Affine(2.0, 0.0, 2797549.5,
        0.0, -2.0, 1195031.0)}

In [ ]:
# What is the elevation range of the Digital Elevation Model?

In [ ]:
#mask no-data values for prettyness:
if dem_src.nodata is not None:
    dem2020 = np.ma.masked_equal(dem2020, dem_src.nodata)

In [ ]:
# Plot elevation using a reasonable range for the elevation. How might you query this if you don't know?
f, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(dem2020,
           vmin=..., 
           vmax=..., 
           cmap='terrain')
cbar = f.colorbar(cax, ax=ax, label='Elevation', shrink=0.5)
ax.set_title('Silvretta elevation')
plt.show()

In [ ]:
# Plot elevation so that the data ends up on the right place on Earth!

# Use the bounds and attributes thereof to define the spatial extent of the DEM. 
bounds = dem_src.bounds
extent = [..., ..., ..., ...] #left, bottom, right, top

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(dem2020,
           extent=...,
           vmin=..., 
           vmax=..., 
           cmap='terrain')
cbar = f.colorbar(cax, ax=ax, label='Elevation', shrink=0.5)
ax.set_title('Silvretta elevation')
plt.show()

### 2. Loading, reprojecting, and manipulating raster data with xdem

xdem is designed to work with elevation data, but it can also be used for other raster data. It's built on rasterio (and by association gdal), but it's a bit more packaged. Load the same dataset again and

1) Use the .plot() method and see how it plots. What do you notice / what is different with regard to using rasterio?
2) Take a look a the xdem [terrain attributes](https://xdem.readthedocs.io/en/stable/terrain.html) and try to plot a hillshade.
3) Load the ice thickness estimate from [Farinotti et al., 2019](https://www.nature.com/articles/s41561-019-0300-3) for Silvretta glacier and plot it. What do you notice with this?
4) Create a plot that shows the ice thickness over the hillshade, including the glacier outline and a colorbar for the ice thickness!
5) Already done? Load a second DEM and create and plot a difference map, showing the glacier outlines as well.


In [ ]:
dem = xdem.DEM(...) #load the dataset

In [ ]:
dem.plot() #notice how it already knows where it is in the world

In [ ]:
# Plot a hillshade. You might want to define a more appropriate colormap. 
... .plot()

In [ ]:
# write the hillshade to file for future use!
hillshade = ...

Let's use xdem to load another raster dataset, namely the ice thickness estimate

In [10]:
rgi_thickness = xdem.DEM('./data/RGI60-11/RGI60-11.00804_thickness.tif') #Ice thickness estimate for Silvretta glacier from a global consensus estimate

In [ ]:
# plot the ice thickness! Can you plot it with the DEM?

What is the problem? Take a look here for a solution (once you have identified the problem ;-) [hint](https://xdem.readthedocs.io/en/stable/gen_modules/xdem.DEM.reproject.html))

In [ ]:
# fix the problem! 
thickness_lv95 = ...

In [ ]:
# load our outline 
outline = ...

In [ ]:
#plot!
f, ax = plt.subplots()
hillshade.plot(ax=ax,
               cmap=...,
               add_cbar=...,
               alpha=...)

thickness_lv95.plot(ax=ax, 
              cmap=..., 
              cbar_title=..., 
              alpha=...)
outline.plot(...)
ax.set_xlim(..., ...)
ax.set_ylim(..., ...)
# add title
# add axes labels

In [ ]:
# loading a second DEM and differencing them

In [ ]:
dem2014 = ...

In [ ]:
dem2014_reproj = ...

In [ ]:
diff = ...

In [ ]:
sgi = ...

In [ ]:
f, ax = plt.subplots()
...
...
ax.set_xlim(..., ...)
ax.set_ylim(..., ...)

### 3. Extracting data from rasters: points, polygons, multi-polygons

Oftentimes, we want to get information from raster datasets that are linked to vectordatasets, e.g., values at points or within on or more polygons. Use the following lines of code to 

1) Extract the minimum, maximum, mean, median, and standard deviation for the sampling points you defined earlier.
2) 

In [ ]:
# sampling a raster dataset at predefined points or extracting information inside a 
# load geojson
sampling_points = gpd.read_file('./data/sampling_points.geojson')

In [ ]:
thickness_rio = thickness_lv95.to_rio_dataset()

In [ ]:
thickness_rio.nodata

In [ ]:
masked_data, masked_transform = mask(thickness_rio, outline.geometry, crop=True, nodata=thickness_rio.nodata)

In [ ]:
# Exclude NaN values from the masked array
masked_data = np.ma.masked_equal(masked_data, thickness_rio.nodata)

In [ ]:
stats = {
    'min': masked_data.min(),
    'max': masked_data.max(),
    'mean': masked_data.mean(),
    'median': np.ma.median(masked_data),
    'std': masked_data.std()
}

print("Zonal Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

In [ ]:
# Can you calculate the volume of ice inside the outline?

In [ ]:
# Create a list to store raster values at points
raster_values_at_points = []

# Sample raster at each point
for index, point in sampling_points.iterrows():
    # Convert point coordinates to pixel coordinates
    pixel_coords = rasterio.transform.rowcol(thickness_rio.transform, point.geometry.x, point.geometry.y)

    # Get raster value at the pixel coordinate
    value = thickness_rio.read(1)[pixel_coords]

    # Add value to the list
    raster_values_at_points.append(value)

# Add raster values to GeoDataFrame
sampling_points['ice_thickness'] = raster_values_at_points

print(sampling_points)

### Now we want to get stats for a series of polygons. Let's say we don't trust the individual point measurements, so we put a buffer around our initial points

In [ ]:
sampling_points['buffered']= sampling_points.geometry.buffer(100)

In [ ]:
buffered_points = sampling_points.copy()
buffered_points = buffered_points.drop(['geometry','ice_thickness'], axis=1)

In [ ]:
buffered_points

In [ ]:
buffered_points = buffered_points.set_geometry('buffered')

In [ ]:
raster_stats_for_polygons = []

for index, row in sampling_points.iterrows():
    # Access the geometry of the current row
    #polygon = sampling_points.geometry[i]
    
    masked_data, masked_transform = mask(thickness_rio, [row.buffered], crop=True, nodata=thickness_rio.nodata)
    
    # Exclude nodata values from the masked array
    masked_data = np.ma.masked_equal(masked_data, thickness_rio.nodata)
    
    # Calculate statistics of the masked array
    stats = {
        'min': masked_data.min(),
        'max': masked_data.max(),
        'mean': masked_data.mean(),
        'median': np.ma.median(masked_data),
        'std': masked_data.std()
    }
    
    # Add statistics to the list
    raster_stats_for_polygons.append(stats)

# Add raster statistics to GeoDataFrame
buffered_points = buffered_points.assign(**{key: [stat[key] for stat in raster_stats_for_polygons] for key in raster_stats_for_polygons[0].keys()})

print(buffered_points)

### 4. Writing raster data

In [ ]:
# Let's load some old that that we previously created

In [ ]:
points_inside = gpd.read_file('data/glathida_inside_sgi.geojson')

In [ ]:
# Extract the bounds of the polygon
minx, miny, maxx, maxy = outline.total_bounds
# Create an empty grid for interpolation
grid_x, grid_y = np.mgrid[minx:maxx:100j, miny:maxy:100j]  # 100j for a 100x100 grid

In [ ]:
points = np.array(list(zip(points_inside.geometry.x, points_inside.geometry.y)))
values = points_inside['THICKNESS'].values

In [ ]:
grid_z = griddata(points, values, (grid_x, grid_y), method='linear') #try also cubic. Which one looks better?

In [ ]:
f, ax = plt.subplots()
#using imshow
ax.imshow(grid_z.T, origin='lower', extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()], aspect='auto', cmap='Blues')
#using contour
#ax.contourf(grid_x, grid_y, grid_z, cmap='Blues')

#add points from measurements
d = plt.scatter(
    points_inside.geometry.x,  # x-coordinates of points
    points_inside.geometry.y,  # y-coordinates of points
    c=points_inside['THICKNESS'],  # values to use for coloring
    cmap='Blues',  # colormap
    s=75,  # size of markers
    edgecolor='none',  # edge color of markers
    alpha=1  # transparency
)
outline.plot(ax=ax, facecolor='None')

In [ ]:
# Now let's save this dataset as a raster
x, y = np.meshgrid(grid_x, grid_y)

In [ ]:
#transform = from_origin(x.min(), y.max(), (x.max()-x.min())/grid_x.shape[1], (y.min()-y.max())/grid_z.shape[0])
pixel_width = (x.max()-x.min())/grid_x.shape[1]
pixel_height = (y.max()-y.min())/grid_z.shape[0]
transform = Affine.translation(x.min(), y.max()) * Affine.scale(pixel_width, -pixel_height)

In [ ]:
transform

In [ ]:
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': -9999,
    'width': grid_z.shape[1],
    'height': grid_z.shape[0],
    'count': 1,
    'crs': 'EPSG:2056',  # Example EPSG code, replace with appropriate CRS
    'transform': transform
}

In [ ]:
with rasterio.open('data/glathida_interpolated.tif', 'w', **metadata) as dst:
    dst.write(np.fliplr(grid_z).astype('float64'), 1)


In [ ]:
thickness_interpolated = xdem.DEM('data/glathida_interpolated.tif')

In [ ]:
thickness_diff = thickness_lv95-thickness_interpolated.reproject(thickness_lv95)

In [ ]:
f, ax = plt.subplots()
#thickness_interpolated.plot(ax=ax, zorder=5, alpha=0.8)
#thickness_lv95.plot(ax=ax, zorder=0, alpha=1)
thickness_diff.plot(ax=ax)

In [ ]:
thickness_diff